# Overview over the exercises

First, the segmentation exercise

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('postal_codes.csv')

Now, data is loaded. Next step, remove all values with non-assigned boroughs:

In [3]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df[df['Neighbourhood']=='Not assigned'] 
# There does not seem to be a case, where the Neighbourhood is not assigned!

,Postal Code,Borough,Neighbourhood


Now, check if we have multiple mentions of postal codes?

In [5]:
len(df['Postal Code'].unique()), df.shape

(103, (103, 3))

No, every postal code is only mentioned once!

In [6]:
df.shape

(103, 3)

# Second part of the exercise

First, try the geocoder thing:

In [8]:
!pip3 install geocoder

     |████████████████████████████████| 98 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 2.1 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=c39db4801472ea05546b8fb7bd1e4f2bb9f84182d9997c2900e97d1dd544ae49
  Stored in directory: /home/ed/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future


In [27]:
import geocoder # import geocoder

n,_=df.shape


df['Latitude']  = [0]*n
df['Longitude'] = [0]*n

for i in range(n):
    postal_code=df['Postal Code'].iloc[i]

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    df['Latitude'].iloc[i] = lat_lng_coords[0]
    df['Longitude'].iloc[i] = lat_lng_coords[1]
    

KeyboardInterrupt: 

It does not finish in reasonable time, so I decided with the other way:

In [29]:
!wget https://cocl.us/Geospatial_data
!mv Geospatial_data Geospatial_data.csv

--2020-12-04 15:12:44--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.194, 169.63.96.176
Connecting to cocl.us (cocl.us)|169.63.96.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-04 15:12:45--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-12-04 15:12:45--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [66]:
gdf = pd.read_csv('Geospatial_data.csv')
print(gdf.columns)
gdf.head()

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [67]:
x=pd.merge(df, gdf, how='left', left_on='Postal Code', right_on='Postal Code', suffixes=[None, "_y"])
x.drop(['Latitude_y', 'Longitude_y'], axis=1, inplace=True)
df=x